### 사전준비

### 환경설정


In [ ]:
# 한글 깨짐을 방지하는 코드
import matplotlib.pyplot as plt

# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
print(f)

# 확인 이후
plt.rc('font', family='Malgun Gothic')

In [ ]:
import pandas as pd
import numpy as np
import re
import io
import os
from tqdm.notebook import tqdm
tqdm.pandas()
from torch.utils.data import Dataset, DataLoader
from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score

# 개인별 데이터 경로 지정
data_path =''

# 분류 라벨
labels_ids = {'01_주장':0, '02_사실':1, '03_판단':2, '04_결론':3}

# How many labels are we using in training.
# This is used to decide size of classification head.
n_labels = len(labels_ids)

### 학습데이터 전처리

In [ ]:
import json
import chardet

def is_json_key_present(json, key):
    try:
        buf = json[key]
    except KeyError:
        return False
    return True

class JsonDataset(Dataset):
    def __init__(self, path):
        if not os.path.isdir(path):
            raise ValueError('Invalid `path` variable! Needs to be a directory')
        
        self.texts = []
        self.labels = []

        for label in ['01.민사', '02.형사', '03.행정']:
            sentiment_path = os.path.join(path, label)

            for (root, directories, files) in os.walk(sentiment_path):
                for file_name in tqdm(files, desc=f'{root} files'):
                    file_path = os.path.join(root, file_name)
                    
                    try:
                        # 인코딩 자동 감지
                        with open(file_path, 'rb') as f:
                            raw = f.read()
                            result = chardet.detect(raw)
                            encoding = result['encoding']
                        
                        with open(file_path, 'r', encoding=encoding) as json_file:
                            json_data = json.load(json_file)
                        
                        # 데이터 추출
                        if is_json_key_present(json_data, "assrs") and is_json_key_present(json_data['assrs'], "acusrAssrs"):
                            for data in json_data['assrs']['acusrAssrs']:
                                self.texts.append(fix_text(data))
                                self.labels.append('01_주장')

                        if is_json_key_present(json_data, "assrs") and is_json_key_present(json_data['assrs'], "dedatAssrs"):
                            for data in json_data['assrs']['dedatAssrs']:
                                self.texts.append(fix_text(data))
                                self.labels.append('01_주장')

                        if is_json_key_present(json_data, "facts") and is_json_key_present(json_data['facts'], "bsisFacts"):
                            for data in json_data['facts']['bsisFacts']:
                                self.texts.append(fix_text(data))
                                self.labels.append('02_사실')

                        if is_json_key_present(json_data, "dcss") and is_json_key_present(json_data['dcss'], "courtDcss"):
                            for data in json_data['dcss']['courtDcss']:
                                self.texts.append(fix_text(data))
                                self.labels.append('03_판단')

                        if is_json_key_present(json_data, "close") and is_json_key_present(json_data['close'], "cnclsns"):
                            for data in json_data['close']['cnclsns']:
                                self.texts.append(fix_text(data))
                                self.labels.append('04_결론')

                    except (UnicodeDecodeError, json.JSONDecodeError, TypeError) as e:
                        print(f"❌ Failed to load or parse {file_path}: {e}")
                        continue

        self.n_examples = len(self.labels)

    def __len__(self):
        return self.n_examples

    def __getitem__(self, item):
        return {
            'text': self.texts[item],
            'label': self.labels[item]
        }

### 모델 초기화

## 학습데이터 LOAD


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# JSON Data 
dataset = JsonDataset(path=data_path)
df = pd.DataFrame.from_records(dataset)
df.groupby('label').size()

df = df.sample(frac=0.05, random_state=123)

df1 = df[df['label'].str.startswith('01')]
df2 = df[df['label'].str.startswith('02')]
df3 = df[df['label'].str.startswith('03')]
df4 = df[df['label'].str.startswith('04')]        

# 전체 데이터에서 train, test 분리 (9 : 1)
df1_train, df1_test = train_test_split(df1, test_size=0.1, random_state=123)
df2_train, df2_test = train_test_split(df2, test_size=0.1, random_state=123)
df3_train, df3_test = train_test_split(df3, test_size=0.1, random_state=123)
df4_train, df4_test = train_test_split(df4, test_size=0.1, random_state=123)

df_train = pd.concat([df1_train,df2_train,df3_train,df4_train])
df_test = pd.concat([df1_test,df2_test,df3_test,df4_test])

In [ ]:
#전체데이터 확인
df.groupby('label').size()

In [ ]:
# 한국어 형태소 분석기 Okt를 사용하기 위해 import
from konlpy.tag import Okt
# Okt의 객체 생성
okt = Okt()

# 문제1: 한국어에서 의미가 거의 없는 단어들을 불용어 처리하기 위해 집합을 만듦(예시: set(['은', '는'])
stop_words = set([])

In [ ]:
# 전처리 함수
def preprocessing(okt, review):
  # 문제2: 한글이 아닌 단어는 지우고 " "으로 대체 
  review_text = 
  # 형태소 분석을 하며 stemming 기법을 사용
  token_li = okt.morphs(review_text, stem=True)
  # 이전 stop_words에 포함되지 않는 단어만 clean_review에 들어갈 수 있게 함
  clean_review = [token for token in token_li if not token in stop_words]
  # 단어들을 ' '기준으로 이어 붙혀서 하나의 string 타입의 변수를 생성함
  clean_review = ' '.join(clean_review)

  # 전처리 완료된 문장을 return
  return clean_review.strip()

In [ ]:
# 문제3: train data와 test data에 대하여 preprocessing (전처리) 함수를 적용


In [ ]:
# 문제4: 전처리 과정에서 모든 문장이 삭제될 가능성이 있으므로(즉, 공백이 존재하게 될 수 있으므로), 해당 데이터(==공백) 제거 및 데이터 개수 확인

In [ ]:
# 문제5: 
# 훈련 데이터 (x, y 형태로 준비)


# 테스트 데이터 (x, y 형태로 준비)


In [ ]:
# 문제6: 
# embedding


In [ ]:
# 문제7: 
# modeling

In [ ]:
# 문제8:
# confusion matrix 출력